In [1]:
import scores.categorical
import xarray as xr

## Overview of Contingency Scores

Many people will be familiar with the following terms: true positives, false negatives, true positive rate. These terms and many more all apply to a contingency table. Producing these scores is inherently a three-step process.

1. If needed, use an "event definition" to convert forecast values to forecasts of events or non-events, as binary information.
2. Produce a contingency object, which holds true positives, true negativates, false positives and false negatives, and supporting various ways to aggregate the data.
3. Calculate the scores as needed from the contingency tables produced in step 2

`scores` provides support for all three steps, plus convenient functions for working efficiently. Most of the scores APIs are functional in nature, however introducing some classes will make contingency scores much easier to calculate and work with. This notebook starts with simple examples and builds up to more complex ones.

## Making binary event data from numerical forecasts and observations

Some forecasts systems inherently produce forecasts of "events". An event might be for example "it will rain", or "a tropical cyclone will occur". These overlay an event definition on the physical phenomena involved. For example, rainfall above a particular precipitation rate might constitute a "heavy rain" event. Something more complex like "a thunderstorm" may involve more complex logic to determine when the event occurs. In this case, the data at hand might be forecasts and observations which are already binary data.

It is also common to be working with a forecast system (including NWP or point forecasts) which generates numerical data rather than categorical data. So, the first step of deriving a contingency score is then to generate the event/non-event data for the forecast and observed conditions. 

Sometimes, the user will have their own way of doing things, and so `scores` will accept such binary data as input.

Sometimes users will want a streamlined way of defining events, and then using scores to generate the binary data and contingency objects together. This notebook demonstrates several approaches, starting with sample real-varying data and deriving the contingency scores.

Consider to begin with the following two tables of forecast values. The values are plausible for either temperatures or precipitation rates.

In [2]:
# Provides a basic forecast data structure in three dimensions
simple_forecast = xr.DataArray(
    [
		[
			[0.9, 0.0,   5], 
			[0.7, 1.4, 2.8],
			[.4,  0.5, 2.3],
		], 
			[
			[1.9, 1.0,  1.5], 
			[1.7, 2.4,  1.1],
			[1.4,  1.5, 3.3],
		], 
	],
	coords=[[10, 20], [0, 1, 2], [5, 6, 7]], dims=["height", "lat", "lon"])

In [3]:
# Within 0.1 or 0.2 of the forecast in all cases except one
# Can be used to find some exact matches, and some close matches
simple_obs = xr.DataArray(
    [
		[
			[0.9, 0.0,   5], 
			[0.7, 1.3, 2.7],
			[.3,  0.4, 2.2],
		], 
			[
			[1.7, 1.2,  1.7], 
			[1.7, 2.2,  3.9],
			[1.6,  1.2, 9.9],
		], 
	],
	coords=[[10, 20], [0, 1, 2], [5, 6, 7]], dims=["height", "lat", "lon"])

The first step of the three-step process is to create event tables for the forecast and observed data. In this case, we will determine an event to have occurred if the value is greater than 1.3. Some users will want to perform this calculation themselves (or may be working with a supplied data set where this has been done already. Others may wish to utilise the classes within scores for reasons of efficiency or re-used. `scores` provides a general class called an "EventOperator". Users can implement this interface themselves for complex scores (we will see an example of this much later), or use one of the in-built operator types. In this case, we will use a simple ThresholdOperator. This can be configured to any of the standard Python operators (e.g. greater-than, less-than or many others). The following code creates a simple "greater" than operator which by default creates events with a threshold of "> 1.3". It can be repeatedly called with different event thresholds, for reasons that will be explained later.

In [4]:
# An event here is defined as a value (e.g. temperature) above 1.3
# The EventThresholdOperator can take a variety of operators from the python "operator" module, or a user-defined function
# The default is operator.gt, which is the same as ">" but in functional form.
event_operator = scores.categorical.ThresholdEventOperator(default_event_threshold=1.3)

In [5]:
# Some users will want to visualise or utilise the event tables directly. This is not necessarily going to be typical, but is demonstrated here for those who wish to step through the process.
forecast_binary, observed_binary = event_operator.make_event_tables(simple_forecast, simple_obs)
print(forecast_binary)

<xarray.DataArray (height: 2, lat: 3, lon: 3)> Size: 18B
array([[[False, False,  True],
        [False,  True,  True],
        [False, False,  True]],

       [[ True, False,  True],
        [ True,  True, False],
        [ True,  True,  True]]])
Coordinates:
  * height   (height) int64 16B 10 20
  * lat      (lat) int64 24B 0 1 2
  * lon      (lon) int64 24B 5 6 7


## Making Contingency Tables

Once the event tables have been made, the next step is to make the contingency manager object. It possible to make the contingency manager from the event operator directly, but we will show it first as separate steps, and then show how to do the two things at once. The contingency manager stores the binary forecast event data, the binary observed event data and the contingency table itself, plus provides functions for aggregating the data along various dimensions, and calculating metrics from that information.

In [6]:
contingency_manager = scores.categorical.BinaryContingencyManager(forecast_binary, observed_binary)
print(contingency_manager)  # Print a view of the contingency table

Contingency Manager (xarray view of table):
<xarray.DataArray (contingency: 5)> Size: 40B
array([ 9,  6,  2,  1, 18])
Coordinates:
  * contingency  (contingency) <U11 220B 'tp_count' 'tn_count' ... 'total_count'


In [7]:
# It is also possible, and briefer, to make the binary contingency table directly from the event operator, as follows:
contingency_manager = event_operator.make_table(simple_forecast, simple_obs)
print(contingency_manager)

Contingency Manager (xarray view of table):
<xarray.DataArray (contingency: 5)> Size: 40B
array([ 9,  6,  2,  1, 18])
Coordinates:
  * contingency  (contingency) <U11 220B 'tp_count' 'tn_count' ... 'total_count'


In [8]:
# It is also possible to retrieve the contingency table as an xarray object
actual_table = contingency_manager.get_table()
actual_table

<xarray.DataArray (contingency: 5)> Size: 40B
array([ 9,  6,  2,  1, 18])
Coordinates:
  * contingency  (contingency) <U11 220B 'tp_count' 'tn_count' ... 'total_count'

In [9]:
# Further, if it turn out you want them, the event tables are still there 
contingency_manager.forecast_events

<xarray.DataArray (height: 2, lat: 3, lon: 3)> Size: 18B
array([[[False, False,  True],
        [False,  True,  True],
        [False, False,  True]],

       [[ True, False,  True],
        [ True,  True, False],
        [ True,  True,  True]]])
Coordinates:
  * height   (height) int64 16B 10 20
  * lat      (lat) int64 24B 0 1 2
  * lon      (lon) int64 24B 5 6 7

## Calculating Contingency Scores

The manager can then be utilised to calculate a wide variety of scores which are based on the contingency table. 

Scores aims to make simple things simple, and complicated things possible. Basic score calculations are simple functions, as follows:

In [10]:
contingency_manager.accuracy()

<xarray.DataArray ()> Size: 8B
array(0.83333333)

In [11]:
contingency_manager.false_alarm_rate()

<xarray.DataArray ()> Size: 8B
array(0.25)

## Tranforming and Computing Large Tables 

There are two kinds of complexity to explore. The first relates to handling very large data sets. Dimensionality can be explored using the "transform" operation on the contingency table. This will also trigger the 'compute' function against the dask arrays which until this point may have been loaded lazily. In this case, we will do a 'transform' with no arguments to trigger the computation of the aggregated contingency table. As such, it may be an expensive operation, but the resultant table is very small and all the relevant scores can be easily calculated as the table is re-used.

In [12]:
computed = contingency_manager.transform()
computed.accuracy()

<xarray.DataArray ()> Size: 8B
array(0.83333333)

## Exploring Dimensionality

The transform function is also utilised to explore the dimensionality of the data. This function accepts preserve_dims and reduce_dims as per the rest of the scores codebase. The intention is to support weights during transformation in the future, but the semantics of using weights in a binary context is unclear, so more time is needed to explore implementation options. For now, this approach is used to examine contingency scores along the data dimensions.

In [13]:
# If it is wanted, the underlying event counts can be accessed
new_manager = contingency_manager.transform(preserve_dims='height')
print(new_manager)

Contingency Manager (xarray view of table):
<xarray.DataArray (contingency: 5, height: 2)> Size: 80B
array([[3, 6],
       [5, 1],
       [1, 1],
       [0, 1],
       [9, 9]])
Coordinates:
  * height       (height) int64 16B 10 20
  * contingency  (contingency) <U11 220B 'tp_count' 'tn_count' ... 'total_count'
